<h1>Dataset Preprocessing to Enable Application of Classical ML Models</h1>

In [106]:
import json

In [107]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [108]:
from model.model import build_preprocessing

In [109]:
# Load configuration file from json in the given folder
with open("config.json", "r") as config_file:
    config = json.load(config_file)

In [110]:
# Set up the list of gestures
with open(config["Paths"]["Gesture list"], "r") as gesture_list:
    gestures = gesture_list.readlines()[0].split(", ")

In [111]:
img_size = config["General parameters"]["Image size"]

In [112]:
train_images, test_images = tf.keras.preprocessing.image_dataset_from_directory("Data",
                                                                                labels="inferred",
                                                                                label_mode="int",
                                                                                class_names=gestures,
                                                                                color_mode="rgb",
                                                                                batch_size=128,
                                                                                image_size=(img_size,
                                                                                            img_size),
                                                                                shuffle=True,
                                                                                seed=42,
                                                                                validation_split=0.2,
                                                                                subset="both")

Found 26351 files belonging to 49 classes.
Using 21081 files for training.
Using 5270 files for validation.


In [113]:
# Set the default preprocessing pipeline if not specified
preprocessing_layers = config["Model"]["Default preprocessing"]
preprocessing_layers = "I,R"

# Build the preprocessing pipeline according to given instructions
preprocessing = build_preprocessing(inp_shape=[img_size,
                                               img_size,
                                               3],
                                    instructions=preprocessing_layers,
                                    name="preprocessing_pipeline")
img_size = 64
resize = tf.keras.layers.Resizing(img_size, img_size)

In [114]:
# Apply the same preprocessing steps as for the CNN but flatten the images
channels = 1 if "G" in preprocessing_layers else 3
train_images = train_images.map(lambda x, y: (tf.reshape(resize(preprocessing(x)), [-1, img_size ** 2 * channels]), y)).unbatch()
test_images = test_images.map(lambda x, y: (tf.reshape(resize(preprocessing(x)), [-1, img_size ** 2 * channels]), y)).unbatch()

In [115]:
# Convert the tf.data.Datasets to numpy arrays containing image & label per sample
train_images = np.array(list(train_images.take(10000).as_numpy_iterator()), dtype=tuple)
test_images = np.array(list(test_images.take(2000).as_numpy_iterator()), dtype=tuple)

In [116]:
# Extract the images and the labels separately
train_X, train_y = np.array([element[0] for element in train_images]), np.ravel(np.vstack([element[1] for element in train_images]))
test_X, test_y = np.array([element[0] for element in test_images]), np.ravel(np.vstack([element[1] for element in test_images]))

In [118]:
# Check memory consumption for the training dataset
print("Training dataset memory consumption: {:.1f}MB".format(train_X.nbytes / (1024 * 1024.0)))

Training dataset memory consumption: 468.8MB


<h1>k-NN Gesture Classification</h1>

In [162]:
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import GridSearchCV

In [ ]:
# Initialize and train the k-NN classifier
knn = KNeighborsClassifier(n_jobs=5)

# Initialize the grid search
parameters = {"p": [1, 2],
              "n_neighbors": [1, 3, 5, 10, 15]}
clf = GridSearchCV(knn, parameters,
                   scoring="accuracy",
                   n_jobs=5,
                   refit=True,
                   verbose=3,
                   cv=3)
clf.fit(train_X, train_y)

Fitting 5 folds for each of 21 candidates, totalling 105 fits


In [157]:
# Evaluate the classifier on the test dataset
knn_predictions = knn.predict(test_X)

print("Accuracy:", accuracy_score(test_y, knn_predictions))
#print(classification_report(test_y,
#                            knn_predictions,
#                            labels=gestures[:-1],
#                            target_names=gestures[:-1]))

Accuracy: 0.9025


In [158]:
np.unique(knn_predictions, return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]),
 array([42, 38, 41, 40, 33, 54, 43, 41, 51, 32, 23, 32, 46, 35, 37, 46, 43,
        38, 37, 42, 50, 44, 41, 45, 50, 54, 38, 56, 42, 35, 46, 43, 42, 44,
        41, 50, 48, 46, 38, 33, 39, 40, 43, 51, 36, 33, 34, 44],
       dtype=int64))

In [159]:
np.unique(test_y, return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]),
 array([41, 40, 37, 38, 31, 54, 43, 47, 46, 44, 22, 39, 35, 32, 37, 47, 45,
        39, 38, 45, 55, 41, 40, 45, 44, 46, 40, 46, 39, 38, 46, 36, 30, 52,
        43, 41, 44, 47, 39, 33, 39, 46, 48, 51, 38, 45, 40, 48],
       dtype=int64))